In [3]:
#import libraries
import os
import random as rd
import tensorflow as tf
import matplotlib.pyplot as plt
import pyBigWig as pbg
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
import matplotlib as mpl
import keras.backend as K
from keras.models import load_model
from tensorflow import keras

In [4]:
#load all chrs
with np.load('/home/florian/projet/r6.16/seq.npz') as f:
    X_2L = f['2L']
    X_2R = f['2R']
    X_3L = f['3L']
    X_3R = f['3R']
    X_4 = f['4']
    X_X = f['X']
    X_Y = f['Y']

In [ ]:
#bw to np
os.chdir('/home/florian/projet/cell_types')
folder=['Astrocyte_like.dedup.no_blacklist.RPGCnormalized.bw',
        'Cortexglia.dedup.no_blacklist.RPGCnormalized.bw',
        'Ensheathingglia.dedup.no_blacklist.RPGCnormalized.bw',
        'Perineurialglia.dedup.no_blacklist.RPGCnormalized.bw',
        'Subperineurialglia.dedup.no_blacklist.RPGCnormalized.bw',
        'T1.dedup.no_blacklist.RPGCnormalized.bw',
        'T2.dedup.no_blacklist.RPGCnormalized.bw',
        'T2a.dedup.no_blacklist.RPGCnormalized.bw',
        'T3.dedup.no_blacklist.RPGCnormalized.bw',
        'T4.dedup.no_blacklist.RPGCnormalized.bw',
        'T5.dedup.no_blacklist.RPGCnormalized.bw',
        'KC_AB.dedup.no_blacklist.RPGCnormalized.bw',
        'KC_ABp.dedup.no_blacklist.RPGCnormalized.bw',
        'KC_G.dedup.no_blacklist.RPGCnormalized.bw']
for cell in folder:
    fichiers=[cell]
    valeurs =[]
    Dic={}
    chr=["2L","2R","3L","3R","4","X","Y"]
    lens=[len(X_2L),len(X_2R),len(X_3L),len(X_3R),len(X_4),len(X_X),len(X_Y)]
    for i in range(len(chr)):
        Dic[chr[i]]=np.zeros((1,lens[i]))
    for file in fichiers:
        bw=pbg.open('/home/florian/projet/scATACseq/'+file)
        array = np.array(())
    for j in chr:
        Dic[j]=Dic[j]+bw.values(j,0,-1,numpy=True)

    os.chdir('/home/florian/projet/cell_types')
    np.savez_compressed('scATACseq_{}'.format(cell),**Dic)

In [1]:
#one hot sequencing function
def ohe(seq):
    seq2=list()
    mapping = {"A":[1, 0, 0, 0],"C": [0, 1, 0, 0],"G":[0, 0, 1, 0],"T":[0, 0, 0, 1]}
    for i in seq:
      seq2.append(mapping[i.upper()] if i.upper() in mapping.keys() else [0, 0, 0, 0]) 
    return np.array(seq2)

In [19]:
#one hot decoding function
def ohd(seq):
    seq2=list()
    mapping = {"[1 0 0 0]":"A", "[0 1 0 0]":"C", "[0 0 1 0]":"G", "[0 0 0 1]":"T"}
    for i in seq:
        seq2.append(mapping[str(i)] if str(i) in mapping.keys() else "x")
    return ''.join(seq2)

In [14]:
#fasta to ohe numpy
f=open('/home/florian/projet/drosoSYNTH/chr4_SynEC_UAS_TATA-yeGFP-NLS.fa','r')
os.chdir('/home/florian/projet/drosoSYNTH')
chr_name = str()
seq=[]
D={}
for line in f:
        line = line.rstrip()
        if line[0] == '>':
                if seq != []:
                        D[chr_name]=ohe(seq)
                chr_name = line[1:5].strip()
                seq = []
        else:
                seq += line
#D[chr_name]=ohe(seq) #if one line only
np.savez_compressed('seqTATA',**D)

In [8]:
for i in ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16"]:
    print(len(np.load('/home/florian/projet/W303/seq.npz')['chr{}'.format(i)]))

252221
844051
341087
1575129
572496
277781
1092105
564939
430780
762303
683383
1084257
929095
793416
1108329
986200


In [13]:
a=np.load('/home/florian/clipped99_MNase.npz')

In [14]:
a.files

['chr01',
 'chr02',
 'chr03',
 'chr04',
 'chr05',
 'chr06',
 'chr07',
 'chr08',
 'chr09',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16']

In [56]:
dicX = {}
with np.load('/home/florian/projet/W303/seq.npz') as f:
    for i in ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16"]:
        dicX['X_{}'.format(i)]=f['chr{}'.format(i)]

In [25]:
X_4_TATA=np.load('/home/florian/projet/drosoSYNTH/seqTATA.npz')['chr4']
X_4_100=np.load('/home/florian/projet/drosoSYNTH/seq100.npz')['chr4']

In [45]:
len(X_4_100),len(X_4_TATA)

(133867, 125243)

In [46]:
model_name='new_cut_weightless'
model2 = load_model('/home/florian/projet/models/'+ model_name +'/'+ model_name+ '.h5', compile=False)

In [47]:
class MyPredSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, batch_size, WINDOW=2001):
        self.x = x_set
        self.batch_size = batch_size
        self.WINDOW = WINDOW
        self.indices = np.arange(len(self.x))
        self.indices=self.indices[self.WINDOW//2:len(self.x)-self.WINDOW//2 -1][::10]

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]
        window_indices = batch_indices.reshape(-1, 1) + np.arange(-(self.WINDOW//2), self.WINDOW//2 + 1).reshape(1, -1)
        batch_x = self.x[window_indices]
        return batch_x

In [48]:
X_chr4_TATA=MyPredSequence(X_4_TATA,256)
X_chr4_100=MyPredSequence(X_4_100,256)

In [ ]:
preds={}
preds['predTATA']=np.concatenate((np.zeros(100),model2.predict(X_chr4_TATA,batch_size=256).ravel(),np.zeros(100)))
preds['pred100']=np.concatenate((np.zeros(100),model2.predict(X_chr4_100,batch_size=256).ravel(),np.zeros(100)))

In [34]:
os.chdir('/home/florian/projet/drosoSYNTH')
np.savez_compressed('preds_droso_synth',**preds)

In [14]:
files = os.listdir('/home/florian/projet/cell_types')
files[3][10:-41]

'Ensheathingglia'

In [125]:
files = os.listdir('/home/florian/projet/cell_types')
os.chdir('/home/florian/projet/cell_types')
D={}
chr=['2L','2R','3L','3R','4','X','Y']
for file in files:
    D[file[10:-41]]=np.load(file)

In [127]:
files = os.listdir('/home/florian/projet/cell_types')
os.chdir('/home/florian/projet/cell_types')
D={}
chr=['2L','2R','3L','3R','4','X','Y']
for file in files:
    D[file[10:-41]]=sum([np.load(file)[i][0].sum() for i in chr])

In [131]:
for file in files:
    print('signal = '+ str(D[file[10:-41]])+' on '+ file[10:-41])

signal = 128972937.85041845 on T3
signal = 127976492.61683974 on KC_AB
signal = 130618559.96108389 on T1
signal = 131536214.65668955 on Ensheathingglia
signal = 129345106.60672629 on T2
signal = 128356589.90209869 on KC_G
signal = 129227740.78160042 on KC_ABp
signal = 129197244.80437791 on T5
signal = 129003451.06918658 on T4
signal = 129451703.74496955 on T2a
signal = 132222184.19734281 on Cortexglia
signal = 131163663.9348709 on Astrocyte_like
signal = 130330790.60432589 on Perineurialglia
signal = 133767980.01546648 on Subperineurialglia


: 

In [ ]:
import csv
os.chdir('/home/florian/projet/scATACseq')
folder=os.listdir('/home/florian/projet/scATACseq')
chr=["2L","2R","3L","3R","4","X","Y"]
for j in chr:
    my_np1=[]
    for cell in folder:
        bw=pbg.open(cell)
        arr=(bw.values(j,0,-1,numpy=True))
        my_np1.append(arr.astype(dtype='float16')[::10])
    my_pd=pd.DataFrame(my_np1,dtype='float16').T
    my_pd.columns=[i[:-len('.dedup.no_blacklist.RPGCnormalized.bw')] for i in folder]
    csv_file = 'signaux{}.csv'.format(j)
    with open(csv_file, mode='w') as f:
        my_pd.to_csv(f)

In [112]:
arrs = [np.random.rand(10000) for i in range(100)]

In [113]:
%%timeit
np.sum([arr.sum() for arr in arrs])

209 µs ± 406 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [114]:
%%timeit
np.sum(np.concatenate(arrs))

418 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [4]:
with np.load('/home/florian/projet/r6.16/seq.npz') as f:
    X_2L = f['2L']


In [6]:
X_2L.shape

(23513712, 4)